In [2]:
def getting_info(name_file,distance_file):

    '''reading name file into list of string'''

    with open(f'{name_file}','r') as readfile:
        cities_list = readfile.read().splitlines()

    '''reading distance file into list of float'''

    with open(f'{distance_file}') as readfile:
        distance_str = readfile.read()
        distance_str = distance_str.split()
    distance_flt = []
    for string in distance_str:
        string = float(string)
        distance_flt.append(string)

    '''creating dictionary (city i,city j):distance from i to j'''

    names_distance = {}
    flag_dist = 0
    for name1 in cities_list:
        for name2 in cities_list:
            names_distance[name1,name2] = distance_flt[flag_dist]
            flag_dist += 1

    return cities_list,names_distance

In [3]:
import random

''' calculate the distance of a route'''

def getting_distance(name_list,dict):
    distance = 0
    for i in range(len(name_list)):
        if i == (len(name_list)-1):
            distance += dict[name_list[i],name_list[0]]
        else:
            distance += dict[name_list[i],name_list[i+1]]
    return round(distance,1)

In [4]:
tsp = []
def tsp_greedy(name_file, distance_file):

    '''creating meaningful city list and dictionary to obtain distance between two cities'''

    list_of_names ,names_distance = getting_info(name_file,distance_file)

    '''set each city as the starting city'''

    for name in list_of_names:

        candidate_route = []
        candidate_route.append(name)

        for i in range(len(list_of_names)):

            '''update the tsp_route if candidate has shorter distance'''

            if len(candidate_route) == len(list_of_names):

                global tsp
                tsp_distance = getting_distance(tsp, names_distance)
                candidate_dist = getting_distance(candidate_route, names_distance)

                if tsp_distance > candidate_dist or tsp_distance == 0:
                    tsp = candidate_route
                    tsp_distance = candidate_dist
            
                '''greedily adding the remaining cities to the candidate'''

            else:
                distance_frm_name = 0
                for key in names_distance.keys():

                    '''prevent invalid solution '''

                    if candidate_route[-1] == key[0] and key[-1] not in candidate_route:
                        if names_distance[key] < distance_frm_name or distance_frm_name == 0:
                            distance_frm_name = names_distance[key]
                            next_closest_city = key[1]

                candidate_route.append(next_closest_city)

    print(f'{tsp}, this is the route and {tsp_distance} is the added distance of all cities. ')
                             
tsp_greedy('thirty_cities_names.txt', 'thirty_cities_dist.txt')
                

['Azores', 'London', 'Paris', 'Berlin', 'Rome', 'Istanbul', 'Cairo', 'Baghdad', 'Moscow', 'Bombay', 'Shanghai', 'Tokyo', 'Guam', 'Manila', 'Melbourne', 'Sydney', 'Honolulu', 'San Francisco', 'Seattle', 'Juneau', 'Chicago', 'New York', 'Montreal', 'New Orleans', 'Mexico City', 'Panama City', 'Santiago', 'Buenos Aires', 'Rio de Janeiro', 'Capetown'], this is the route and 527.0 is the added distance of all cities. 


In [5]:
def tsp_mutation(name_file, distance_file):
    
    '''INITIALIZATION'''

    '''creating parent route and dictionary of city names and distances'''

    parent_route, names_distance = getting_info(name_file,distance_file)
    parent_distance = getting_distance(parent_route,names_distance)
    stagnation = 0
    solution_s = {}

    '''TERMINATE'''

    '''if encoutering 5 stagnations, escape the while loop'''
    while stagnation < 5:

        ''' assuming best offspring is parent route and distance'''

        best_offspring_route = parent_route
        best_offspring_distance = parent_distance

        '''MUTATE'''

        for offspring in range(10):

            '''randomly choose 2 city from parent route'''

            picked_city1 = random.choice(parent_route)

            '''avoid having duplicate city names'''

            ''' making copy for adjustion'''

            best_offspring_route_copy = parent_route.copy()
            best_offspring_route_copy.remove(picked_city1)
            best_offspring_route_copy.append(picked_city1)
            candidate_offspring = getting_distance(best_offspring_route_copy,names_distance) 

            '''SELECT'''

            '''update the best offspring if meeting the condition'''

            if candidate_offspring < best_offspring_distance:
                best_offspring_route = best_offspring_route_copy
                best_offspring_distance = candidate_offspring

        '''encounter stagnation and mutate 3 times away from it'''

        if best_offspring_distance == parent_distance:

            '''update stagnation and solution_s dict'''

            stagnation += 1
            solution_s[tuple(best_offspring_route)] = best_offspring_distance 

            '''INITIALIZE'''

            stagnation_parent = best_offspring_route

            '''mutate three times'''
            
            for s in range(3):

                picked_city2 = random.choice(stagnation_parent)

                stagnation_copy = stagnation_parent.copy()
                stagnation_copy.remove(picked_city2)
                stagnation_copy.append(picked_city2)

                stagnation_distance = getting_distance(stagnation_copy,names_distance)

            '''update parent route and distance accepting the worse solution after 3 times of mutation'''

            parent_route = stagnation_copy
            parent_distance = stagnation_distance
        
        else:
            parent_route = best_offspring_route
            parent_distance = best_offspring_distance
    
    '''find the shortest route amongst stagnation and its distance'''
   
    for key in solution_s.keys():
        if solution_s[key] == min(solution_s.values()):
            print(f'For mutation, TSP is {key} and {solution_s[key]} in distance')

tsp_mutation('thirty_cities_names.txt','thirty_cities_dist.txt')

For mutation, TSP is ('Azores', 'Berlin', 'Bombay', 'Guam', 'Honolulu', 'Juneau', 'Manila', 'Melbourne', 'Mexico City', 'Montreal', 'New Orleans', 'Panama City', 'Rio de Janeiro', 'San Francisco', 'Seattle', 'Shanghai', 'Sydney', 'Tokyo', 'Moscow', 'Cairo', 'Paris', 'Rome', 'Istanbul', 'Baghdad', 'Chicago', 'New York', 'Buenos Aires', 'Capetown', 'Santiago', 'London') and 1085.0 in distance


In [8]:
def backtracking(route,cities,distances,total):

    if len(route) == 7:

        '''calculating the distance from the last city and the first city to complete a city round'''

        first = route[0]
        last = route[-1]
        returning_to = distances[first,last]
        final_total = total + returning_to

        global best_route
        global best_distance

        ## ivan note:
        ## i couldn't figure out how to put this into the function
        ## tsp backtracking having a "global" value for the route and distance
        ## i felt like putting so many values into the actual backtracking function was...
        ## too much. so i just made it super simple into a global function

        if final_total < best_distance:
             
            best_route = route
            best_distance = final_total

    else:

        for i in range(len(cities)):
                    
            copy_route = route.copy()
            copy_rem = cities.copy()
            copy_total = total

            copy_route.append(copy_rem[i]) 
            copy_rem.remove(copy_rem[i])

            if len(copy_route) >= 2:
                    
                curr_city = copy_route[-2]
                next_city = copy_route[-1]
                dist_between = distances[curr_city,next_city]
                copy_total += dist_between

            ## to linh:
            ## b/c of the way how the recursion occurs
            ## i can't just put the code in the if statement (seen above)
            ## it doesn't work because of how lists works so whatever

            if len(copy_route) <= 7:
                    
                    backtracking(copy_route,copy_rem,distances,copy_total)

best_route = []
best_distance = 1000000

## ivan note:
## best_distance is so high because of the reason that i was too lazy
## b/c of the way the backtracking program works
## this is probably the easiest and least amount of work needed to be done

def tsp_backtracking(cities_names,cities_distances):

    cities, distances = getting_info(cities_names,cities_distances)

    backtracking([],cities,distances,0)

    global best_route
    global best_distance

    print(f"best route was: {best_route}")
    print(f"distance of best route: {round(best_distance,1)}")

tsp_backtracking('seven_cities_names.txt', 'seven_cities_dist.txt')


best route was: ['Beta', 'Zeta', 'Delta', 'Gamma', 'Epsilon', 'Alpha', 'Eta']
distance of best route: 106.4
